In [24]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
import numpy as np
from qiskit.visualization import array_to_latex
import numpy as np
from numpy import pi
from qiskit import QuantumCircuit, QuantumRegister, Aer
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info import Statevector
from qiskit.circuit import Gate
from qiskit.visualization import array_to_latex

In [25]:
def qft_rotations(qc, n):
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qb in range(n):
        qc.cp(pi/2**(n-qb), qb, n)
    qft_rotations(qc, n)

def swap_qb(qc, n):
    for qb in range(n//2):
        qc.swap(qb, n-1-qb)
    return qc    

def F(n):
    qc = QuantumCircuit(n, name='QFT')
    qft_rotations(qc, n)
    swap_qb(qc, n)
    gate = qc.to_gate()
    return qc

In [26]:
def iqft_rotations(qc,n,i):
    if i == n-2: 
        qc.h(i+1)
        return qc
    i += 1
    qc.h(i)
    for qb in reversed(range(i+1)):
        qc.cp(-np.pi/2**(i+1-qb),qb,i+1)
    iqft_rotations(qc,n,i)
    
def swap_IQFT_qb(n):
    qc = QuantumCircuit(n, name='IQFT')
    swap_qb(qc,n)
    iqft_rotations(qc,n,-1)
    return qc

In [27]:
n = 3

In [29]:
qft_circ = F(n)
inv_qft_circ = swap_IQFT_qb(n)
test_circ =qft_circ.compose(inv_qft_circ)

In [30]:
print("Composed Circuit:")
print(test_circ)

Composed Circuit:
                                          ┌───┐      ┌───┐               »
q_0: ──────■──────────────────────■───────┤ H ├─X──X─┤ H ├─■─────────────»
           │                ┌───┐ │P(π/2) └───┘ │  │ └───┘ │P(-π/2) ┌───┐»
q_1: ──────┼────────■───────┤ H ├─■─────────────┼──┼───────■────────┤ H ├»
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │  │                └───┘»
q_2: ┤ H ├─■────────■───────────────────────────X──X─────────────────────»
     └───┘                                                               »
«                              
«q_0: ───────────■─────────────
«                │             
«q_1: ─■─────────┼─────────────
«      │P(-π/2)  │P(-π/4) ┌───┐
«q_2: ─■─────────■────────┤ H ├
«                         └───┘


In [31]:
array_to_latex(Operator(test_circ.to_gate()))

<IPython.core.display.Latex object>